In [2]:
!createdb -h fonduer-postgres-dev -U postgres jkracht
#!dropdb -h fonduer-postgres-dev -U postgres jkracht

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
import subprocess
import logging
from pprint import pprint

subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'nltk'])

from nltk.corpus import wordnet as wn
import nltk
nltk.download('wordnet')

from fonduer.candidates.matchers import LambdaFunctionMatcher, Union, Intersect, RegexMatchSpan

from fonduer import Meta, init_logging
from fonduer.parser.preprocessors import HTMLDocPreprocessor
from fonduer.parser import Parser
from fonduer.parser.models import Document, Sentence

PARALLEL = 2
ATTRIBUTE = "jkracht"
conn_string = f'postgresql://postgres@fonduer-postgres-dev:5432/{ATTRIBUTE}'

init_logging(log_dir="logs")

session = Meta.init(conn_string).Session()

# Document parser
docs_path = "data/test_collection2/"
doc_preprocessor = HTMLDocPreprocessor(docs_path)

corpus_parser = Parser(session, structural=True, lingual=True)
corpus_parser.apply(doc_preprocessor, parallelism=PARALLEL)


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[2023-05-31 14:30:35,796][INFO] fonduer.meta:49 - Setting logging directory to: logs/2023-05-31_14-30-35
[2023-05-31 14:30:35,810][INFO] fonduer.meta:135 - Connecting user:postgres to fonduer-postgres-dev:5432/jkracht
[2023-05-31 14:30:35,845][INFO] fonduer.meta:162 - Initializing the storage schema
[2023-05-31 14:30:37,268][INFO] fonduer.utils.udf:67 - Running UDF...


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from fonduer.parser.models import Document, Section, Table, Cell, Paragraph, Sentence, Figure, Caption
print(f"Num Docs: {session.query(Document).count()}")

print(f"Num Sections: {session.query(Section).count()}")
print(f"Num Tables: {session.query(Table).count()}")
print(f"Num Cells: {session.query(Cell).count()}")
print(f"Num Sentences: {session.query(Sentence).count()}")
print(f"Num Paragraphs: {session.query(Paragraph).count()}")

print(f"Num Figures: {session.query(Figure).count()}")
print(f"Num Captions: {session.query(Caption).count()}")

Num Docs: 2
Num Sections: 4
Num Tables: 0
Num Cells: 0
Num Sentences: 3470
Num Paragraphs: 2591
Num Figures: 40
Num Captions: 0


In [8]:
# mention
from fonduer.candidates.models import mention_subclass
from fonduer.candidates import MentionExtractor
from fonduer.candidates import MentionNgrams
from fonduer.candidates.mentions import MentionSentences, MentionCaptions, MentionParagraphs, MentionDocuments, MentionTables
from fonduer.utils.data_model_utils.textual import get_neighbor_sentence_ngrams, get_left_ngrams
Task = mention_subclass("Task")


list_of_headlines = [
                     #"Acknowledgements", "Acknowledgement", "acknowledgements", "acknowledgement", 
                     "Contributions", "Contribution", "contribution", "contributions",
                     "Author Contributions",
                     #"Credits", "Credit", "credits", "credit",
                     "Überschrift 1"]
    
# def mention_span_in_acknowledments_matches_verb(mention):
#     span_string = mention.get_span()

#     try:
#         # get last paragraphs first sentence (headline of the paragraph)
#         headline_of_last_paragraph = session.query(Paragraph).get(mention.sentence.paragraph_id-1).sentences[0].text

#         # check if last headline is listed to extract mentions of
#         #if headline_of_last_paragraph in list_of_headlines:
#         if any(option in headline_of_last_paragraph for option in list_of_headlines):

#             #test if span is a verb
#             for word in wn.synsets(span_string):
#                 if word.pos() == "v": # and word.name().split(".")[0] == span_string.lower():
#                     return True # case: span is a ver in a wanted paragraph

#             return False # case: span is not a verb
        
#         else:
#             return False # case: span not in wanted paragraph
#     except:
#         return False # case: no prior paragraph


def mention_get_verb(mention):
    """
    function checks
    """
    span_string = mention.get_span()
    doc_id = mention.sentence.paragraph.document_id
    para_pos = mention.sentence.paragraph.position
    try:
        headline = session.query(Paragraph).filter(Paragraph.position >= para_pos-10, Paragraph.position <= para_pos-1, Paragraph.document_id == doc_id)
        for h in headline:
            x = h.sentences[0].text
            if any(option.lower() in x.lower() for option in list_of_headlines):
                return True
                # #test if span is a verb
                # for word in wn.synsets(span_string):
                #     if word.pos() == "v": # and word.name().split(".")[0] == span_string.lower():
                #         # print("1")
                #         return True # case: span is a ver in a wanted paragraph
                # # print("2")
                # return False # case: span is not a verb
        
        else:
            # print("3")
            return False # case: span not in wanted paragraph
    except:
        # print("4")
        return False
        

# matcher_task1 = LambdaFunctionMatcher(func = mention_span_in_acknowledments_matches_verb)
matcher_task2 = LambdaFunctionMatcher(func = mention_get_verb)

task_space = MentionSentences()

matchers_task = Intersect(
    # matcher_task1,
    matcher_task2

)

docs = session.query(Document).all()

mention_extractor = MentionExtractor(
    session,
    [Task],
    [task_space], #task_sentences #task_space
    [matchers_task],
    parallelism=PARALLEL,
)

mention_extractor.apply(docs, parallelism=PARALLEL, clear=True)
print(
    f"Number of Tasks: {session.query(Task).count()}",
)


[2023-05-31 14:39:29,165][INFO] fonduer.candidates.mentions:467 - Clearing table: task
[2023-05-31 14:39:29,167][INFO] fonduer.utils.udf:67 - Running UDF...


  0%|          | 0/2 [00:00<?, ?it/s]

Number of Tasks: 0


In [4]:
para_pos = 11
a = list(session.query(Paragraph).filter(Paragraph.position <= 11, Paragraph.position >= 1, Paragraph.document_id == 118064))
for b in a:
    print(b.sentences[0].text)
#list(session.query(Paragraph).filter(Paragraph.position == 10, Paragraph.document_id == 118064))[0].sentences[0].text

In [7]:
for i in session.query(Paragraph).all():
    print(i.sentences[0].text)
    print(i.position)
    print("xxxxx")


A modular RNA interference system for multiplexed gene regulation | bioRxiv
0
xxxxx
Skip to main content
1
xxxxx
Home
2
xxxxx
About
3
xxxxx
Submit
4
xxxxx
ALERTS / RSS
5
xxxxx
Search for this keyword
6
xxxxx
Advanced Search
7
xxxxx
New Results
8
xxxxx
A modular RNA interference system for multiplexed gene regulation
9
xxxxx
Ari Dwijayanti, Marko Storch, Guy-Bart Stan, Geoff S. Baldwin
10
xxxxx
doi:
11
xxxxx
Ari Dwijayanti
12
xxxxx
1CNRS@CREATE, 1 Create Way, #8-01 Create Tower, Singapore 138602
13
xxxxx
Find this author on Google Scholar
14
xxxxx
Find this author on PubMed
15
xxxxx
Search for this author on this site
16
xxxxx
Marko Storch
17
xxxxx
2London Biofoundry, Translation & Innovation Hub, White City Campus, Imperial College London, London W12 0BZ, UK
18
xxxxx
3Imperial College Centre for Synthetic Biology, Imperial College London, London SW7 2AZ, UK
19
xxxxx
Find this author on Google Scholar
20
xxxxx
Find this author on PubMed
21
xxxxx
Search for this author on this site
22
xx

In [ ]:
a = 0
for mention in session.query(Task).all():
    print(mention.context.get_span())
    #print("\n")
    if a > 100:
        break
    a += 1

In [216]:
a = session.query(Paragraph).get(3325)

print(dir(a))

['__bool__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [32]:
for a in session.query(Paragraph).all():
    print(a.position)
    print(a.document_id)
    print(a.id)
    print(a.sentences[0].text)
    print("----")

0
66205
66209
21
----
1
66205
66211
22 23 24 25 26!
----
2
66205
66213
35
----
3
66205
66215
36 37 38 39 40.
----
0
66206
66210
1
----
1
66206
66212
2 3 4 5 6!
----
2
66206
66214
15
----
3
66206
66216
16 17 18 19 20.
----


In [229]:
a = session.query(Task).all()
for b in a:
    print(b.id)


# b = a.sentences[0].text

get_neighbor_sentence_ngrams(b)

1869
1870


<generator object get_neighbor_sentence_ngrams at 0x7f456fb490d0>

In [214]:
a = session.query(Sentence).get(29011)
a.text
a.get_neighbor_sentence_ngrams(window = 1, n= 1)

'Bla Blub.'

In [202]:
from fonduer.candidates.models import candidate_subclass
from definitions.mentions.mention_subclasses import NameFull, NameAbbr, Task

NameFullAbbr = candidate_subclass("NameFullAbbr", [NameFull, NameAbbr])
NameFullTask = candidate_subclass("NameFullTask", [NameFull, Task])

from fonduer.candidates import CandidateExtractor
from fonduer.parser.models import Document
from pipeline.utils import split_documents, get_session, PARALLEL
from pipeline.throttler.NameFullTask import NameFullTask_throttlers


candidate_extractor = CandidateExtractor(
                                session,
                                [NameFullAbbr,NameFullTask], #NameFullAbbr,
                                throttlers=NameFullTask_throttlers,
                            )

# for split, docs in enumerate(split_documents(session)):
for split, docs in enumerate(session.query(Document).all()): 
    docs = [docs]
    print(f"\nSplit {split}: {len(docs)} documents")
    candidate_extractor.apply(docs, split=split, parallelism=PARALLEL)
    print(f"Split {split}: Number of NameFull + NameAbbrv candidates:", session.query(NameFullAbbr).filter(NameFullAbbr.split==split).count())
    print(f"Split {split}: Number of NameFull + Task candidates:", session.query(NameFullTask).filter(NameFullTask.split==split).count())



AttributeError: 'NoneType' object has no attribute 'sentence_id'

In [37]:
# Testbereich

def mention_span_in_acknowledments_matches_verb(mention):
    # read in span/word
    span_string = mention

    # Task(SpanMention("RefWorks Tagged", sentence=61332, chars=[0,14], words=[0,1]))
    
    for word in span_string.split(): # mention.get_span()[0]
        #print(word)
        word_filtered = word.replace("!","")
        try:
            if wn.synsets(word_filtered)[0].pos() == "v":
                #print(word)
                return word_filtered
        except:
            pass
    return False

a = "I played soccer!"

In [38]:
mention_span_in_acknowledments_matches_verb(a)

'played'

In [109]:
wn.synsets("want")[0].pos()

'n'

In [124]:
for a in wn.synsets("want"):
    print(a.pos())
    print(a.name().split(".")[0])

n
privation
n
lack
n
need
n
wish
v
desire
v
want
v
want
v
want
v
want


In [ ]:
    sentence_of_span = mention.sentence.text
    #print(sentence_of_span)
    #x = mention._get_table()
    # print(mention.sentence.paragraph_id)
    # #print(mention.document_id)
    # print("x")
    # print(mention.sentence.paragraph_id-1)
    #print(mention.sentence.paragraph_id-1)
    # a = session.query(Paragraph).get(mention.sentence.paragraph_id-1)
    #print(a.sentences[0].text)

In [186]:

# if any in 

a = ["abc", "bbb", "adf"]
headline_of_last_paragraph = "Acknowledgment"

if any(option in headline_of_last_paragraph for option in list_of_headlines):
    print("T")
else:
    print("F")

F


In [15]:
a = "Auto"
b = "auto"
print(b.islower())

True


# Idee
Throttler
    - alle candidates mit task mention die Groß geschrieben sind aussortieren
        - Fall Satzanfang: sollten keine Verben stehen???

# Probleme
1. get_neighbor_sentence_ngrams
bekomme mit 'get_neighbor_sentence_ngrams' keine Paragraph Sprünge hin
    - Alternativweg: n_max riesig groß wählen, den ngram durchiterieren, bis eine andere paragraph_id vorkommt
    - weiter: ersten satz dieser paragraph_id auf 'acknowledgment' checken
    - dann auf verb testen

2. PARALLEL > 1 Problem für paragraph_id
- Ursprungsproblem: Paragraph_ids werden anders verteilt bei PARALLEL > 1
    - vorherige id ist nicht id-1

3. Bedenken: zu viel im Throttler
- sortiert alles außerhalb des gewünschten paragraphen aus
- check auf verb (vielleicht erst im throttler)


# Getan
Matcher geschrieben, der bei Parallel = 1 funktioniert
- guckt ob eine Form von 'acknowledgment', 'contribution', etc. im ersten Satz (Überschrift) des vorherigen Paragraphen steht
- checkt ob eingelesenes Wort ein Verb ist
    - importiert nltk.corpus
    - synset-wortlist für jeden span, checken ob pos() == "v" (word ist ein Verb) -> match
